In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

/Users/rituc/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
dir_path = "~/workspace/kaggle_toxic_comment_classification/data/"

In [3]:
df = pd.read_csv(dir_path+"train.csv")
test_df = pd.read_csv(dir_path+"test.csv")

In [4]:
len(df)

95851

In [5]:
train_df, val_df = train_test_split(df, test_size=0.25)

In [6]:
print len(train_df)
print len(val_df)

71888
23963


In [7]:
test_df.head() 

,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==


In [8]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
41749,436743242923,"""\n\nThanks for always being so speedy in reve...",0,0,0,0,0,0
60790,633516523668,Ernie Smith writes to the Wikipdiad Web manage...,0,0,0,0,0,0
72659,757918443637,"""\n\nActually, Wikipedia:Removing warnings was...",0,0,0,0,0,0
28021,290606788986,"""\nGeez. And from looking at his user page he ...",0,0,0,0,0,0
30555,317906712972,"""\n\nAs you may recall, you've helped us with ...",0,0,0,0,0,0


In [12]:
cols = train_df.columns[2:]

In [21]:

for col in cols:
    print col, sum(train_df[col]), "=>", sum(train_df[col])*100.0/train_df.shape[0]

toxic 6992 => 9.726240819051858
severe_toxic 715 => 0.9946027153349655
obscene 3870 => 5.383374137547296
threat 242 => 0.336634765190296
insult 3616 => 5.030046739372357
identity_hate 621 => 0.8638437569552637


In [16]:
train_df[cols].describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,71888.000000,71888.000000,71888.000000,71888.000000,71888.000000,71888.000000
mean,0.097262,0.009946,0.053834,0.003366,0.050300,0.008638
std,0.296317,0.099233,0.225691,0.057923,0.218566,0.092542
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
text = train_df["comment_text"]

In [14]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text)
X_train_counts.shape

(71888, 117367)

In [15]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

/Users/rituc/anaconda2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(71888, 117367)

In [16]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(71888, 117367)

In [90]:
# train_df[labels]

In [17]:
labels = train_df.columns[2]

In [18]:
labels

'toxic'

In [19]:
from sklearn.naive_bayes import MultinomialNB

In [20]:
clf = MultinomialNB().fit(X_train_tfidf, train_df[labels])

In [21]:
len(train_df)

71888

In [22]:
clf.predict(X_train_tfidf)

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
from sklearn.pipeline import Pipeline

In [24]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [25]:
text_clf

Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [26]:
clf.predict(X_train_tfidf)

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
train_df["predicted"] = clf.predict(X_train_tfidf)
train_df["diff"] = train_df["predicted"] - train_df[labels] 
train_df["diff"].value_counts()

/Users/rituc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/rituc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


 0    66095
-1     5776
 1       17
Name: diff, dtype: int64

In [31]:
# val_df["predicted"] = clf.predict(X_train_tfidf)
# val_df["diff"] = train_df["predicted"] - train_df[labels] 
# val_df["diff"].value_counts()

In [29]:
X_train_tfidf

<71888x117367 sparse matrix of type '<type 'numpy.float64'>'
	with 3127665 stored elements in Compressed Sparse Row format>

In [32]:
# val_df["predicted"] = clf.predict(X_train_tfidf)